# Gather Energy Data from LLMs

Test energy usage with these models:
- **GPT-4o-mini** (OpenAI)
- **Claude 3 Sonnet** (Anthropic) 
- **Mistral Large** (Together.ai)

In [1]:
import pandas as pd

In [2]:
# load the dataset
data = pd.read_json("data/sample.json")

In [3]:
# show sample of the data
data.head()

,prompt_text
0,how can identity protection services help prot...
1,Beside OFAC's selective sanction that target t...
2,You are the text completion model and you must...
3,The sum of the perimeters of three equal squar...
4,What is the type of the variables in the follo...


In [4]:
# Get energy data from LLM API calls
import codecarbon
import openai
import anthropic
import os
from dotenv import load_dotenv


In [5]:
# Set up LLM clients
load_dotenv()

clients = {}
clients['openai'] = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
clients['anthropic'] = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
clients['mistral'] = os.getenv("MISTRAL_API_KEY")


In [6]:
# Track energy during API calls
# TODO: Energy tracking function
# TODO: Log energy data


In [7]:
# Make API calls to each model
# TODO: OpenAI function
# TODO: Anthropic function
# TODO: Google function
# TODO: Together.ai function


In [8]:
# Test energy with sample prompts
# TODO: Run tests
# TODO: Save energy data
